In [133]:
from itertools import chain, combinations, permutations

def get_optimal_bunny_list(bunnies1, bunnies2):
    if len(bunnies1) < len(bunnies2):
        return bunnies2
    if len(bunnies1) > len(bunnies2):
        return bunnies1
    if bunnies1[0] < bunnies2[0]:
        return bunnies1 
    if bunnies1[0] > bunnies2[0]:
        return bunnies2
    if sum(bunnies1) > sum(bunnies2):
        return bunnies2 
    if sum(bunnies1) < sum(bunnies2):
        return bunnies1
    return bunnies2

def detect_negative_time_loop(times):
    n = len(times)
    for i in range(n):
        nums = list(range(i)) + list(range(i + 1, n))
        for l in chain.from_iterable(combinations(nums, r) for r in range(1, n)):
            for path in permutations(l):
                pos = i
                t = 0
                for step in list(path) + [i]:
                    t += times[pos][step]
                    pos = step
                if t < 0:
                    return True
    return False

def get_minimum_times_to_bulkhead(times):
    mins = {}
    n = len(times)
    for i in range(n - 1):
        nums = list(range(i)) + list(range(i + 1, n - 1))
        min_time = float('inf')
        for l in chain.from_iterable(combinations(nums, r) for r in range(n - 1)):
            for path in permutations(l):
                pos = i
                t = 0
                for step in list(path) + [n - 1]:
                    t += times[pos][step]
                    pos = step
                if t < min_time:
                    min_time = t
        mins[i] = min_time
    return [mins[i] for i in range(n - 1)]

def detect_repeated_cycle(path):
    path = [cell[0] for cell in path]
    if path.count(path[-1]) < 3:
        return False
    path.reverse()
    second_idx = path[1:].index(path[0]) + 1
    if len(path) > second_idx * 2 and path[1: second_idx + 1] == path[second_idx + 1: second_idx * 2 + 1]:
        return True 
    return False

def get_bunnies(stack, n):
    positions = set([cell[0] for cell in stack])
    positions.discard(0)
    positions.discard(n - 1)
    return [i - 1 for i in sorted(list(positions))]

def solution(times, time_limit):
    n = len(times)
    if detect_negative_time_loop(times):
        return list(range(n - 2))
    best_bunnies = []
    min_times = get_minimum_times_to_bulkhead(times)
    stack = [(0, time_limit)]
    x = 0
    prev_stacks = set()
    while len(stack) > 0 and x < 100:
        print(stack)
        print(best_bunnies)
        prev_stacks.add(tuple(stack[:]))
        x += 1
        curr_pos, curr_t = stack[-1][0], stack[-1][1]
        for next_pos in range(n):
            if next_pos == curr_pos:
                continue
            potential_t = curr_t - times[curr_pos][next_pos]
            if tuple(stack + [(next_pos, potential_t)]) in prev_stacks:
                print(f'rejected {next_pos} because it was in prev stacks')
                continue
            if next_pos != n - 1 and potential_t < min_times[next_pos]:
                print(f'rejected {next_pos} because of time constraints')
                continue
            if detect_repeated_cycle(stack + [(next_pos, potential_t)]):
                print(f'rejected {next_pos} because of repeated cycle')
                continue
            stack.append((next_pos, potential_t))
            if next_pos > 0 and next_pos < n - 1:
                print(f'moving to pos {next_pos}')
                bunnies = get_bunnies(stack, n)
                if bunnies == list(range(n - 2)):
                    return bunnies 
                best_bunnies = get_optimal_bunny_list(best_bunnies, bunnies)
            break
        if stack[-1] == (curr_pos, curr_t):
            stack.pop()
    return best_bunnies


inputs = [
    ([
        [0, 2, 2, 2, -1],   # 0 --> 4 --> 2 --> 4 --> 3 --> 4
        [9, 0, 2, 2, -1], 
        [9, 3, 0, 2, -1],
        [9, 3, 2, 0, -1],
        [9, 3, 2, 2, 0],
    ], 1),
    ([
        [0, 1, 1, 1, 1], 
        [1, 0, 1, 1, 1], 
        [1, 1, 0, 1, 1], 
        [1, 1, 1, 0, 1], 
        [1, 1, 1, 1, 0]
    ], 3),
    ([
        [0, 1, 1, 1, 5, 4, 7],
        [3, 0, 1, 1, 5, 4, 7], 
        [4, 1, 0, 1, 5, 1, -1], 
        [8, 1, 1, 0, 2, 4, 7], 
        [9, 1, 1, 1, 0, 4, 7], 
        [9, 1, 2, 1, 2, 0, 0], 
        [1, 1, 1, 1, 6, 4, 0], 
    ], 3), # 
    ([
        [0, 3, -1],
        [1, 0, 1 ],
        [9, 2, 0 ],
    ], 2), 
    ([
        [0,  5,  6,  7,  -2 ], 
        [9,  0,  34, 5,  8  ], 
        [4,  200, 0,  39, 40], 
        [246,9,  99, 0,  243], 
        [2,  49, 1,  68, 0  ]
    ], 1), 
    ([
        [0,  1  ,100,1  ,1  ,1  ,100],
        [100,0  ,100,100,100,100,1  ], 
        [100,100,0  ,100,100,100,100],  
        [100,100,100,0  ,100,100,100], 
        [100,100,100,100,0  ,1  ,100], 
        [1  ,100,100,100,100,0  ,100], 
        [100,100,100,100,100,100,0  ], 
    ], 8), # 
    ([
        [0  ,100,100,100,-5 ,100,5  ],  
        [100,0  ,1  ,100,100,100,100],  
        [1  ,100,0  ,100,100,100,100],  
        [100,100,100,0  ,100,100,5  ],  
        [100,2  ,100,100,0  ,100,4  ],  
        [100,100,100,100,100,0  ,100],  
        [2  ,100,100,5  ,5  ,100,0  ],  
    ], 20), 
    ([
        [0  ,1  ,100,100,100], 
        [-2 ,0  ,100,100,100], 
        [100,100,0  ,100,100], 
        [100,100,100,0  ,100], 
        [100,100,100,100,0  ],  
    ], 1), 
    ([
        [0  ,1  ,1  ,100], 
        [-1 ,0  ,100,100], 
        [-1 ,100,0  ,100], 
        [100,100,100,0  ], 
    ], 200), 
]

for i in range(8, 9):
    # print(detect_negative_time_loop(inputs[i][0]))
    print(solution(inputs[i][0], inputs[i][1]))

[(0, 200)]
[]
moving to pos 1
[(0, 200), (1, 199)]
[0]
[(0, 200), (1, 199), (0, 200)]
[0]
moving to pos 1
[(0, 200), (1, 199), (0, 200), (1, 199)]
[0]
rejected 0 because of repeated cycle
moving to pos 2
[0, 1]
